# Mapping

[Mappings](https://nexus-forge.readthedocs.io/en/latest/interaction.html#mapping) are pre-defined and declarative rules that encode the logic on how to transform a specific data source into [Resources](https://nexus-forge.readthedocs.io/en/latest/interaction.html#resource) that eventually conform to targeted schemas supported by the configured [Model](https://nexus-forge.readthedocs.io/en/latest/interaction.html#modeling). 

This notebook specifically demonstrates the `DictionaryMapping` wich is based on a JSON structure that represent a targeted structure along with Python code that will apply desired transformations on the data source.

In [1]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [2]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [3]:
from kgforge.core import Resource

In [4]:
from kgforge.specializations.mappings import DictionaryMapping

## Data

In [5]:
scientists = [
    {
        "id": 123,
        "name": "Marie Curie",
        "gender": "female",
        "middle_name": "Salomea",
    },
    {
        "id": 456,
        "name": "Albert Einstein",
        "gender": "male",
        "middle_name": "(missing)",
    },
]

## Mapping data to a targeted template

### basics

Note: DemoModel and RdfModel schemas have not been synchronized yet. The following cell is to be run with DemoModel.

In [6]:
forge.types()

Managed entity types:
   - Activity
   - Contribution
   - Dataset
   - Entity
   - Ontology
   - Person


In [7]:
forge.template("Contribution")

{
    id: ""
}


In [8]:
mapping_simple = DictionaryMapping("""
    type: Contribution
    agent:
    {
        type: Person
        name: x.name
    }
""")

In [9]:
resources_simple = forge.map(scientists, mapping_simple)

In [10]:
print(resources_simple[0])

{
    type: Contribution
    agent:
    {
        type: Person
        name: Marie Curie
    }
}


### missing values

In [11]:
mapping_na = DictionaryMapping("""
    type: Contribution
    agent:
    {
        type: Person
        name: x.name
        additionalName: x.middle_name
    }
""")

In [12]:
print(forge.map(scientists[1], mapping_na))

{
    type: Contribution
    agent:
    {
        type: Person
        additionalName: (missing)
        name: Albert Einstein
    }
}


In [13]:
print(forge.map(scientists[1], mapping_na, na="(missing)"))

{
    type: Contribution
    agent:
    {
        type: Person
        name: Albert Einstein
    }
}


### multiple mappings

In [14]:
mapping_person = DictionaryMapping("""
    id: forge.format("identifier", "persons", x.id)
    type: Person
    name: x.name
""")

In [15]:
mapping_association = DictionaryMapping("""
    type: Contribution
    agent: forge.format("identifier", "persons", x.id)
""")

In [16]:
resources_graph = forge.map(scientists, [mapping_person, mapping_association])

In [17]:
print(resources_graph[0])

{
    id: https://kg.example.ch/persons/123
    type: Person
    name: Marie Curie
}


In [18]:
print(resources_graph[1])

{
    type: Contribution
    agent: https://kg.example.ch/persons/123
}


### managed mappings

Note: RdfModel doesn't implement managed mappings operations yet. Please use DemoModel for this section.

In [ ]:
forge.sources()

In [ ]:
forge.mappings("scientists-database")

In [ ]:
mapping = forge.mapping("Contribution", "scientists-database")

In [ ]:
resources = forge.map(scientists, mapping, na="(missing)")

In [ ]:
type(resources)

In [ ]:
type(resources[0])

In [ ]:
print(mapping)

In [ ]:
print(resources[0])

## Managing mappings

Note: RdfModel doesn't implement managed mappings operations yet. Please use DemoModel for this section.

In [24]:
filepath = "mappings/scientists-database/DictionaryMapping/Contribution.hjson"

### saving

In [ ]:
mapping.save(filepath)

In [26]:
filepath

'mappings/scientists-database/DictionaryMapping/Contribution.hjson'

### tracking & sharing changes

In [ ]:
# ! cd mappings

In [ ]:
# ! git add Contribution.hjson

In [ ]:
# ! git commit -m "Add Association mapping"

In [ ]:
# ! git push

### loading

In [27]:
loaded = DictionaryMapping.load(filepath)

In [ ]:
# loaded == mapping

In [ ]:
# ! rm -R ./mappings/